In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Vertex AI特征存储（遗留）：流式导入SDK

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/feature_store_streaming_ingestion_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/feature_store/feature_store_streaming_ingestion_sdk.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/feature_store/feature_store_streaming_ingestion_sdk.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 在Vertex AI工作台中打开
    </a>
  </td>
</table>

注意：此笔记本已在以下环境中进行测试：

- Python 版本 = 3.9

## 概述

本笔记本演示了如何在 Vertex AI 功能存储（Legacy）的 SDK 层中使用流式导入。

了解有关 [Vertex AI 功能存储](https://cloud.google.com/vertex-ai/docs/featurestore) 的更多信息。

### 目标

在本教程中，您将学习如何使用Vertex AI SDK中的`write_feature_values`方法，将`Pandas DataFrame`中的特征导入到Vertex AI Feature Store（Legacy）中。

本教程使用以下Google Cloud ML服务和资源：

- Vertex AI Feature Store（Legacy）


执行的步骤包括：

- 创建一个特征商店。
- 为您的特征商店创建一个新的实体类型。
- 将`Pandas DataFrame`中的特征值导入到特征商店中的实体类型中。

### 数据集

这个笔记本使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的企鹅数据集。该数据集具有以下特征：`culmen_length_mm`、`culmen_depth_mm`、`flipper_length_mm`、`body_mass_g`、`species`和`sex`。

### 成本

本教程使用了 Google Cloud 的计费组件：

* Vertex AI

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预估用量生成成本估算。

## 安装

安装以下所需的软件包以执行这个笔记本。

In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform\
                         google-cloud-bigquery\
                         numpy\
                         pandas\
                         db-dtypes\
                         pyarrow -q

只有Colab：取消注释以下单元格以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建账户时，您将获得 $300 的免费信用额度，用于支付计算/存储花费。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行此笔记本，则需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下操作：
* 执行 `gcloud config list`。
* 执行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明进行操作。

1. 顶点AI工作台
* 无需操作，因为您已经通过认证。

2. 本地 JupyterLab 实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 給您的服務帳戶授予 Cloud Storage 權限。

### UUID

如果您正在进行现场教程，您可能会使用共享测试帐户或项目。为了避免在创建的资源上发生名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 导入库

In [ ]:
import numpy as np
import pandas as pd
from google.cloud import aiplatform, bigquery

### 初始化用于Python的Vertex AI SDK

为您的项目初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

下载并准备数据

In [ ]:
def download_bq_table(bq_table_uri: str) -> pd.DataFrame:
    # Remove bq:// prefix if present
    prefix = "bq://"
    if bq_table_uri.startswith(prefix):
        bq_table_uri = bq_table_uri[len(prefix) :]

    table = bigquery.TableReference.from_string(bq_table_uri)

    # Create a BigQuery client
    bqclient = bigquery.Client(project=PROJECT_ID)

    # Download the table rows
    rows = bqclient.list_rows(
        table,
    )
    return rows.to_dataframe()

In [ ]:
BQ_SOURCE = "bq://bigquery-public-data.ml_datasets.penguins"

# Download penguins BigQuery table
penguins_df = download_bq_table(BQ_SOURCE)

### 准备数据

要写入特征存储的特征值可以采用一个由`WriteFeatureValuesPayload`对象组成的列表形式，一个Python`dict`的形式是

`{entity_id : {feature_id : feature_value}, ...},`

或者一个pandas`Dataframe`，其中`index`列包含唯一实体ID字符串，每个剩余的列代表一个特征。在这个笔记本中，由于您使用pandas `DataFrame`来摄取特征，我们将索引列的数据类型转换为`string`以便用作`实体ID`。

In [ ]:
# Prepare the data
penguins_df.index = penguins_df.index.map(str)

In [ ]:
# Remove null values
NA_VALUES = ["NA", "."]
penguins_df = penguins_df.replace(to_replace=NA_VALUES, value=np.NaN).dropna()

## 创建特征存储并定义模式

Vertex AI特征存储（Legacy）按以下顺序分层组织资源：

`特征存储 -> 实体类型 -> 特征`

在您可以将数据导入Vertex AI特征存储（Legacy）之前，您必须先创建这些资源。

了解更多关于[Vertex AI特征存储](https://cloud.google.com/vertex-ai/docs/featurestore)。

### 创建特征存储

您可以使用`aiplatform.Featurestore.create`函数来创建特征存储，需要提供以下参数：

* `featurestore_id (str)`: 特征存储的ID，将成为`featurestore`资源名称的最终组成部分。该值需在项目和地点内保持唯一。
* `online_store_fixed_node_count`: 在线服务资源的配置。
* `project`: 要在其中创建`EntityType`的项目。如果未设置，则将使用在`aiplatform.init`中设置的项目。
* `location`: 要创建`EntityType`的地点。如果未设置，则将使用在`aiplatform.init`中设置的地点。
* `sync`: 是否同步执行此创建操作。

In [ ]:
FEATURESTORE_ID = f"penguins_{UUID}"

penguins_feature_store = aiplatform.Featurestore.create(
    featurestore_id=FEATURESTORE_ID,
    online_store_fixed_node_count=1,
    project=PROJECT_ID,
    location=REGION,
    sync=True,
)

#### 确认特征仓库已创建
通过运行以下代码块检查特征仓库是否已成功创建。

In [ ]:
fs = aiplatform.Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=PROJECT_ID,
    location=REGION,
)
print(fs.gca_resource)

### 创建实体类型

实体类型是一组语义相关的特征集合。您可以根据您的用例中相关的概念来定义自己的实体类型。例如，电影服务可能有`movie`和`user`等实体类型，它们将相关特征分组对应于电影或用户。

在这里，您可以使用`create_entity_type`创建一个名为`penguin_entity_type`的实体类型，参数如下：
* `entity_type_id（str）`：用于`EntityType`的ID，将成为`EntityType`资源名称的最终组成部分。该值在特征存储中必须是唯一的。
* `description`：`EntityType`的描述。

In [ ]:
ENTITY_TYPE_ID = f"penguin_entity_type_{UUID}"

# Create penguin entity type
penguins_entity_type = penguins_feature_store.create_entity_type(
    entity_type_id=ENTITY_TYPE_ID,
    description="Penguins entity type",
)

验证实体类型是否已创建
运行以下代码块，并检查实体类型是否成功创建。

In [ ]:
entity_type = penguins_feature_store.get_entity_type(entity_type_id=ENTITY_TYPE_ID)

print(entity_type.gca_resource)

### 创建特性
特性是实体类型的可测属性或特征。例如，`企鹅` 实体类型具有诸如 `鳍长度（毫米）` 和 `体重（克）` 等特性。可以在每个实体类型中创建特性。

创建特性时，您需要指定其值类型，比如 `DOUBLE` 和 `STRING`。该数值会确定可以为特定特性导入什么类型的数值。

了解更多关于[特性值类型](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.featurestores.entityTypes.features)。

In [ ]:
penguins_feature_configs = {
    "species": {
        "value_type": "STRING",
    },
    "island": {
        "value_type": "STRING",
    },
    "culmen_length_mm": {
        "value_type": "DOUBLE",
    },
    "culmen_depth_mm": {
        "value_type": "DOUBLE",
    },
    "flipper_length_mm": {
        "value_type": "DOUBLE",
    },
    "body_mass_g": {"value_type": "DOUBLE"},
    "sex": {"value_type": "STRING"},
}

你可以使用 `create_feature` 或 `batch_create_features` 来创建特征。在这里，为了方便起见，你已将所有特征配置添加到一个变量中，因此我们使用 `batch_create_features`。

In [ ]:
penguin_features = penguins_entity_type.batch_create_features(
    feature_configs=penguins_feature_configs,
)

### 写特征到特征存储

使用`write_feature_values` API将特征写入特征存储，需要以下参数：

* `instances`: 要写入特征存储的特征数值，可以采用`WriteFeatureValuesPayload`对象列表、Python字典或pandas数据框的形式。

流式导入已在Vertex AI SDK中的**preview**命名空间下引入。在这里，您将从企鹅数据集创建的pandas数据框作为`instances`参数传递。

了解更多关于[流式导入API](https://github.com/googleapis/python-aiplatform/blob/e6933503d2d3a0f8a8f7ef8c178ed50a69ac2268/google/cloud/aiplatform/preview/featurestore/entity_type.py#L36)。

In [ ]:
penguins_entity_type.preview.write_feature_values(instances=penguins_df)

## 阅读回写的特征

等待几秒钟让写入传播，然后在线阅读以确认写入是否成功。

In [ ]:
ENTITY_IDS = [str(x) for x in range(100)]
penguins_entity_type.read(entity_ids=ENTITY_IDS)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除您用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除您在此教程中创建的各个资源。

In [ ]:
penguins_feature_store.delete(force=True)